## Combining information from simulation and experiment in a GP

Condensed, fewer visializations

In [1]:
from __future__ import print_function, division

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import GPy
import json
from scipy import signal

from utilities import *
import os
import pickle 
import random

# from PhysicsAPI.physics_simulation import Simulator

import bayesian_optimization
import acquisition_functions

from funcs import *

In [2]:
# Set paths
rpath = './exp1_trials/'
# rpath = './SampleTrials/'
setups = [f[:-5] for f in os.listdir(rpath) if f[0] != '.' and f[0] != 'T' and f[0:5] != 'intro' and f[-4:] == 'json']
setups = np.unique(np.array(setups))
# l = setups.size
wpath = rpath.replace('trials', 'modeling')
wpath = wpath.replace('Trials', 'modeling')

Epath = wpath + 'ESruns/only_expt/' 

In [3]:
# Parameters are sd_smooth, noise_exp, noise_err, factor entropy_thresh, p_diff_thresh and probability_thresh

def runAnalysis(setupfn, factor, index, noise_err_fac, entropy_thresh, p_thresh, dyna, 
                store = True, run = True, RAverbose = True):
    
    # acquire data
    noisy_fun, true_fun = raster(rpath, wpath, setupfn, n_disc, n_approx)
    td = round(100*np.mean(true_fun))
    sd = round(100*np.mean(noisy_fun))    
    if RAverbose:
        print("Percentage of ground truth in :", td)
        print("Percentage of simulations in :", sd)
    
    
    # process data
    sd_smooth = 3
    sim_logit, avg_s = dprocess(n_disc, sd_smooth, noisy_fun)
    true_logit, avg_t = dprocess(n_disc, sd_smooth, true_fun)
    error_logit = sim_logit - true_logit
    
    # setup model
    noise_exp = 0.00001 ** 2
#     noise_err = (0.125*np.std(error_logit)) ** 2
    noise_err = (noise_err_fac*np.std(error_logit)) ** 2
    tm = tradeoff_model(sim_logit, true_logit, sd_smooth, noise_exp, noise_err)

    # Some test data
    X, Y = startXY(tm)

    # Define a GP model with the correct noise
    m = GPy.models.GPHeteroscedasticRegression(X, Y, tm.kernel)
    m['.*het_Gauss.variance'] = tm.get_het_noise(m.X)

    #Run ES
    af = acquisition_functions.EntropySearch(m)
    opt = bayesian_optimization.BayesianOptimizer(m, af)

#     opt, count, n_e, ans, yans, tans, ytans, = runES(tm, factor, thresh, m, af, opt, verbose = False, pverbose = RAverbose)
    if run:
        if dyna: opt = runES(tm, factor, entropy_thresh, p_diff_thresh, p_thresh, avg_t, m, af, opt, dyna = True, verbose = True, pverbose = RAverbose)
        else: opt = runES(tm, factor, entropy_thresh, p_diff_thresh, p_thresh, avg_t, m, af, opt, verbose = True, pverbose = RAverbose)
    
        # store runs
        if store:
            
#             print("storing participant {} setup {}".format(index, setupfn))
            
            if dyna:
                sourcefile = Epath + 'psweep_dyna_trial_data.csv'
            else:
                sourcefile = Epath + 'psweep_full_trial_data.csv'

            with open(sourcefile, 'a') as outfile:
                Ns, Ne, fx, _ = getData_ESruns(opt = opt)
                outfile.write('{}, {}, {}, {}, {}, {}, {}, {}, {}\n'.format(
                    index, setupfn, factor, Ne, Ns, fx, noise_err_fac, entropy_thresh, p_thresh))
                
               
               
                
                
#     return (tm, opt, avg_s, avg_t)
    return


# # GP-prior
# plot(tm, opt, prior=True)
# plt.show()

In [4]:
# Generate N_part more "participants"
# set parameter values and which_model (can convert to taking command line arguments)

p_diff_thresh = 0
N_part = 1

for noise_err_fac in [0.5]:
    for factor_low in [0.0000001]:
        factor_high = 2.0*factor_low
        for entropy_thresh in [0.1]:
            for p_thresh in [0.1]:
                for dyna in [False]:

                    print("Running: dyna = {}, fac_low = {}, fac_high = {}, entr_thresh = {}, noise_err = {}, p_thresh = {}".format(
                        dyna, factor_low, factor_high, entropy_thresh, noise_err_fac,  p_thresh))


                    part_IDs = [int(x) for x in np.random.uniform(size = N_part)*10000000]
                    for s in setups:
                        count = 0
                        while count < N_part:
                            index = part_IDs[count]
                            count += 1
                            All_vals = runAnalysis(s, factor_low, index, noise_err_fac, entropy_thresh, p_thresh, dyna, 
                                                   RAverbose = False)
                            All_vals = runAnalysis(s, factor_high, index, noise_err_fac, entropy_thresh, p_thresh, dyna, 
                                                   RAverbose = False)

Running: dyna = False, fac_low = 1e-07, fac_high = 2e-07, entr_thresh = 0.1, noise_err = 0.5, p_thresh = 0.1
Experiment
Prob of success thresh reached  [ 0.56669507] 0.0
/n ********** /n
Experiment
Prob of success thresh reached  [ 0.56669507] 0.0
/n ********** /n
Experiment
Prob of success thresh reached  [ 0.42449218] 178.181818182
/n ********** /n
Experiment
Prob of success thresh reached  [ 0.96747185] 203.636363636
/n ********** /n
Experiment
Prob of success thresh reached  [ 0.77703759] 320.0
/n ********** /n
Experiment
Prob of success thresh reached  [ 0.14534414] 178.181818182
/n ********** /n
Experiment
Prob of success thresh reached  [ 0.20123254] 29.0909090909
/n ********** /n
Experiment
Experiment
Prob of success thresh reached  [ 0.30771521] 112.727272727
/n ********** /n
Experiment
Prob of success thresh reached  [ 0.35612439] 14.5454545455
/n ********** /n
Experiment
Prob of success thresh reached  [ 0.35612439] 345.454545455
/n ********** /n
Experiment
Prob of success t